In [1]:
# train.py
from copy import deepcopy
import os
import torch
import numpy as np
import warnings
warnings.filterwarnings(action='ignore')

from data_loader import load_data_1m
from feature_calculations import (
    resample_data, calculate_MA_data, calculate_ema_bollinger_bands, calculate_rsi,
    calculate_macd, calculate_stochastic_oscillator, calculate_adx, calculate_atr,
    calculate_obv, calculate_williams_r, base_feature_fn, cyclic_encode_fn, log_transform
)
from strategies import BB_fitness_fn
from dataset import make_dataset, replace_nan_with_zero
from train_functions import inference, fitness_fn, generation, generation_valid, generation_test
from train_functions import loss_cut_fn, calculate_same, calculate_diff, calculate_hold, calculate_now_profit, after_forward

from Prescriptor import Prescriptor
from Evolution.crossover import UniformCrossover, WeightedSumCrossover
from Evolution.mutation import MultiplyNormalMutation, MultiplyUniformMutation, AddNormalMutation, AddUniformMutation, ChainMutation, FlipSignMutation
from Evolution.selection import RouletteSelection, TournamentSelection
from Evolution import Evolution

In [5]:

# Test Case 3: All 'take' actions with entry_pos='short'
# -------------------------------------------------------
diff_prob = torch.tensor([
    [0.1, 0.2, 0.7, 0.0, 0.3, 0.4],
    [0.2, 0.3, 0.5, 0.0, 0.2, 0.3],
    [0.3, 0.4, 0.3, 0.0, 0.1, 0.2]
])
pos_list = np.array(['long', 'long', 'long'])
price_list = np.array([150.0, 250.0, 350.0])
leverage_ratio = np.array([1.5, 2.5, 3.5])
enter_ratio = np.array([0.4, 0.5, 0.6])
profit = np.array([15.0, 25.0, 35.0])
entry_pos = 'short'
curr_close = 145.0

pos_out, price_out, leverage_out, enter_out, profit_out = calculate_diff(
    diff_prob, pos_list, price_list, leverage_ratio, enter_ratio, profit, entry_pos, curr_close
)

print(pos_out)
print(price_out)
print(leverage_out)
print(enter_out)
print(profit_out)


['hold' 'hold' 'shor']
[ -1.  -1. 145.]
[-1.         -1.         52.49791718]
[-1.         -1.          0.54983395]
[ 12.94  -27.625 -88.21 ]


In [3]:
(200 - 105) / 200*100*3*0.5 - 3*0.1*0.6

71.07